# Discussion 09 Notebook

This notebook is an accompaniment to the associated discussion worksheet handout.

# Section II: Entity Resolution

## Database Setup

In [ ]:
!psql -h localhost -c 'DROP DATABASE IF EXISTS disc09'
!psql -h localhost -c 'CREATE DATABASE disc09'

%reload_ext sql
%sql postgresql://127.0.0.1:5432/disc09

In [ ]:
!psql postgresql://127.0.0.1:5432/disc09 <disc09.sql

## Initial Exploration

Let us first take a look at the content of the database.

In [ ]:
%sql SELECT * FROM albums;

In [ ]:
%sql SELECT * FROM sales;

There is one significant barrier to performing data analysis: joins. Although we have cleaned data locally, within each table, we have not cleaned up the connections between these tables such as the strings we may want to join on.

In our dataset, we'll notice that simply joining on equality of names will result in missing data.

### 1. Let's try it out: write a query to join the tables using strict equality

In [ ]:
%%sql
-- your code here

We see that we are missing a lot of data, which could cause trouble in computations later on.

## Distance Functions on Strings
The Levenshtein distance function can be helpful to find strings that are _similar_ but not identical. This algorithm computes the minimal number of insertions, deletions, and mutations. We can apply this to our dataset.

### 2. First, let's compute the Levenshtein distance between every pair of names in the two tables, sorted by this distance.

In [ ]:
%%sql
-- your code here

You'll notice that it's a very close boundary between the strings we want to match and those we don't. In reality, a clustering-based approach would be better suited for this scenario, but let's keep going with distance since that's a lot easier to implement.

We can use `< 10` as our threshold for matching strings.

### 3. Create a materialized view `joined_sales` that joins the tables using Levenshtein distance

Make sure to have the following columns in the view:
- `name`: name of the album
- `artist`: name of the artist
- `day`: day of sales
- `count`: number of sale

In [ ]:
%%sql
-- your code here

# Section III: Data Granularity

## Initial Exploration

In [6]:
%sql SELECT * FROM nodes ORDER BY tax_id LIMIT 5;

Running query in 'postgresql://127.0.0.1:5432/disc07'

5 rows affected.

tax_id,parent,rank
1,1,no rank
2,131567,superkingdom
6,335928,genus
7,6,species
9,32199,species


In [7]:
%sql SELECT * FROM names ORDER BY tax_id LIMIT 5;

Running query in 'postgresql://127.0.0.1:5432/disc07'

5 rows affected.

tax_id,name_txt
1,all
1,root
2,Bacteria
2,bacteria
2,eubacteria


### 1. Write a SQL query to find the node representing the Animalia kingdom.

In [ ]:
%%sql
-- your code here

### 2. Let us drill down into the Animalia kingdom. First, find all children nodes of the Animalia kingdom.

In [ ]:
%%sql
-- your code here

### 3. Next, find the names of these nodes, along with the names of their parents.

In [ ]:
%%sql
-- your code here

In [21]:
%%sql
SELECT nodes.tax_id, name_txt
FROM nodes, names
WHERE nodes.tax_id = names.tax_id
LIMIT 10;

Running query in 'postgresql://127.0.0.1:5432/disc07'

10 rows affected.

tax_id,name_txt
1,all
1,root
2,Bacteria
2,bacteria
2,eubacteria
2,Monera
2,Procaryotae
2,Prokaryotae
2,Prokaryota
2,prokaryote


In [12]:
%%sql
WITH edges AS (
SELECT parent.tax_id AS pid, parent.rank as prank,
child.tax_id AS cid, child.rank as crank
FROM nodes parent, nodes child
WHERE child.parent = parent.tax_id
)
SELECT pid, edges.prank, json_agg(parent.name_txt),
       array_agg(parent.name_txt), cid, edges.crank,
json_agg(child.name_txt)
FROM names parent, names child, edges
WHERE parent.tax_id = edges.pid
AND child.tax_id = edges.cid
AND parent.tax_id = 33208
GROUP BY pid, edges.prank, cid, edges.crank;

Running query in 'postgresql://127.0.0.1:5432/disc07'

2 rows affected.

pid,prank,json_agg,array_agg,cid,crank,json_agg_1
33208,kingdom,"['Animalia', 'animals', 'metazoans', 'Metazoa', 'multicellular animals', 'Animalia', 'animals', 'metazoans', 'Metazoa', 'multicellular animals', 'Animalia', 'animals', 'metazoans', 'Metazoa', 'multicellular animals', 'Animalia', 'animals', 'metazoans', 'Metazoa', 'multicellular animals']","['Animalia', 'animals', 'metazoans', 'Metazoa', 'multicellular animals', 'Animalia', 'animals', 'metazoans', 'Metazoa', 'multicellular animals', 'Animalia', 'animals', 'metazoans', 'Metazoa', 'multicellular animals', 'Animalia', 'animals', 'metazoans', 'Metazoa', 'multicellular animals']",6040,phylum,"['Parazoa', 'Parazoa', 'Parazoa', 'Parazoa', 'Parazoa', 'Porifera', 'Porifera', 'Porifera', 'Porifera', 'Porifera', 'sponges', 'sponges', 'sponges', 'sponges', 'sponges', 'sponges', 'sponges', 'sponges', 'sponges', 'sponges']"
33208,kingdom,"['Animalia', 'animals', 'metazoans', 'Metazoa', 'multicellular animals']","['Animalia', 'animals', 'metazoans', 'Metazoa', 'multicellular animals']",6072,clade,"['Eumetazoa', 'Eumetazoa', 'Eumetazoa', 'Eumetazoa', 'Eumetazoa']"


### 4. You will find there are many synonym names for the same phylum with the same tax_id. Aggregate them up by grouping by tax_id.

In [ ]:
%%sql
-- your code here

### 5. Challenge: How can we drill down one more layer? What if we want to get the names of all the classes under the Animalia kingdom?

In [ ]:
%%sql
-- your code here